<a href="https://colab.research.google.com/github/glaucio-rocha/ciencia_de_dados/blob/main/MVP/MVP_Sprint_III_02_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install duckdb duckdb-engine jupysql


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00


In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 50
pd.options.display.max_columns = None

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import duckdb
%sql duckdb:///:memory:


Importa os arquivos preparados pelo script 01

In [ ]:
%%sql

-- efetua um drop nas tabelas votacao, resultado, despesas, pib_variacao e semiarido caso existam

DROP TABLE IF EXISTS votacao;
DROP TABLE IF EXISTS resultado;
DROP TABLE IF EXISTS despesas;
DROP TABLE IF EXISTS pib_variacao;
DROP TABLE IF EXISTS semiarido;

CREATE TABLE votacao as
    SELECT * FROM "../dados/parquet/votacao_secao_2020_SE.parquet";

CREATE TABLE resultado as
    SELECT * FROM "../dados/parquet/votacao_candidato_munzona_2020_SE.parquet";

CREATE TABLE despesas as
    SELECT * FROM "../dados/parquet/despesas_contratadas_candidatos_2020_SE.parquet";

CREATE TABLE pib_variacao as
    SELECT * FROM "../dados/parquet/pib2020_mun_var128.parquet";

CREATE TABLE semiarido as
    SELECT * FROM "../dados/parquet/semiarido.parquet";






### Transformação de dados
Unifica as informações de municipios e uma unica tabela.
1. Aos dados de variação do PIB é adicionada a informação se o município pertence ao semiárido (transformando o código em SIM/NÃO)
2. A tabela é limitada aos municípios do estado de Sergipe
3. Os dados da geometria dos municípios foram mantidos na tabela

In [ ]:
%%sql result <<

SELECT p.codmun as id_mun, p.municipio as nm_mun, p.var128 as variacao,
      (CASE WHEN s.cd_semiari = 1 THEN 'SIM' ELSE 'NÃO' END) as semiarido,
      p.geometry
  FROM PIB_VARIACAO p, semiarido s
  WHERE uf = 28
  and p.codmun = s.cd_mun



In [ ]:
result.to_parquet('../dados/parquet/dim_municipios.parquet', engine='pyarrow', compression='snappy')

Limpeza da tabela de resultados da eleição

In [ ]:
%%sql  result <<

SELECT ANO_ELEICAO,	TP_ABRANGENCIA,	SG_UF,	SG_UE,	NM_UE,	CD_MUNICIPIO,	NM_MUNICIPIO,	NR_ZONA,
       DS_CARGO,	SQ_CANDIDATO,	NR_CANDIDATO,	NM_CANDIDATO,	NM_URNA_CANDIDATO,	NM_SOCIAL_CANDIDATO,
       DS_SITUACAO_CANDIDATURA,	DS_DETALHE_SITUACAO_CAND, TP_AGREMIACAO,	NR_PARTIDO,	SG_PARTIDO,	NM_PARTIDO,
       QT_VOTOS_NOMINAIS,	NM_TIPO_DESTINACAO_VOTOS,	QT_VOTOS_NOMINAIS_VALIDOS,	CD_SIT_TOT_TURNO,	DS_SIT_TOT_TURNO
  FROM resultado r



In [ ]:
result.to_parquet('../dados/parquet/dim_candidatos.parquet', engine='pyarrow', compression='snappy')

Limpeza da tabela de despesas dos candidatos na eleição
1. Exclusão de dados sensíveis como: CPF e CNPJ dos candidatos e prestadores de serviço

In [ ]:
%%sql result <<

select ANO_ELEICAO,	DS_ELEICAO,	TP_PRESTACAO_CONTAS, DT_PRESTACAO_CONTAS, SG_UF,	SG_UE,	NM_UE,	DS_CARGO,
       NR_CANDIDATO, NM_CANDIDATO, NR_PARTIDO, SG_PARTIDO, NM_PARTIDO, CD_TIPO_FORNECEDOR,	DS_TIPO_FORNECEDOR,
       DS_TIPO_DOCUMENTO, DS_ORIGEM_DESPESA, DT_DESPESA, VR_DESPESA_CONTRATADA
  from despesas


In [ ]:
result.to_parquet('../dados/parquet/fato_despesa.parquet', engine='pyarrow', compression='snappy')